In [1]:
from ipywidgets import AppLayout, Button, Layout,Label,HTML,Image, HBox,VBox,Tab,Accordion,IntSlider,Textarea,Text,DatePicker,Output,Dropdown,IntProgress
from keplergl import KeplerGl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import requests
from shapely.geometry import Point
import datetime
import numpy
from h3 import h3
from shapely.geometry import Polygon
import ipyaggrid

%run ClearConfig.py
map_1= KeplerGl(height=800,config=clearconfig) 
#df={}
#df = pd.read_csv('C:/Users/cyeager/Desktop/UMBC/DATA 601/Final Project/BPD_Part_1_Victim_Based_Crime_Data (1).csv',sep=',')
class APISetup:
    def __init__(self):
        #style = {'description_width': '200px',}
        #layout = {'width:':'4000px'}
        self.button = Button(description='Fetch Data', button_style='success', layout=Layout(height='auto', width='auto'))
        #self.button.on_click(on_click)
        #self.button.on_click(on_clickdata2)
        self.button2 = Button(description='Expand Map', button_style='info', layout=Layout(height='auto', width='auto'))
        
        #%matplotlib widget
        #self.out=Output()
        self.header_button = HBox([HTML(value='<div style="max-height: 120px;width: 9%; float:left;"><img src="bpd_logo.png" width="90%"></div><div style="width: 80%;min-height: 140px; float:left;background:navy;"><h1 style="color:gold;padding: 10px 0;text-align:center;font-size: 250%">Baltimore Police Department Case Connection App</h1></div><div style="max-height: 120px;width: 9%; float:right;"><img src="bpd_logo.png" width="90%"></div>'),], layout=Layout(height='160px', width='100%'))

        self.start = DatePicker(disabled=False,layout=Layout(width='auto'))
        self.end = DatePicker(disabled=False,layout=Layout(width='auto'))
        self.left_button = VBox([Label(value="Pick a Start Date"),self.start,Label(value="Pick a End Date"),self.end,self.button,])
        #self.center_button = VBox([map_1],border='1px solid black')
        #self.right_button = VBox([])
        #self.footer_button = VBox([self.out], layout=Layout(height='400px', width='100%'))

    def on_click(self, function):
        self.button.on_click(function)
    def get_layout(self):
        return AppLayout(header=self.header_button,
              left_sidebar=self.left_button,
              center=self.center_button,
              right_sidebar=self.right_button,
              footer=self.footer_button,
              )
    def QueryIndex(self):
            get1='"https://egis.baltimorecity.gov/egis/rest/services/GeoSpatialized_Tables/Part1_Crime/FeatureServer/0/query?where=CrimeDateTime >= '
            get2= ' AND CrimeDateTime <= '
            get3= ' &outFields=*&returnIdsOnly=true&outSR=4326&f=json"'
            #print(get1+"'"+str(start.value)+"'"+get2+"'"+str(end.value)+"'"+get3)
            QueryIndex= get1+"'"+str(self.start.value)+"'"+get2+"'"+str(self.end.value)+"'"+get3    
            return QueryIndex
        
    def get_URL(self):
            get1='"https://egis.baltimorecity.gov/egis/rest/services/GeoSpatialized_Tables/Part1_Crime/FeatureServer/0/query?where=CrimeDateTime >= '
            get2= ' AND CrimeDateTime <= '
            get3= ' &outFields=CrimeDateTime,Description,CrimeCode,District,Neighborhood,Latitude,Longitude,Premise&returnGeometry=false&outSR=4326&f=json"'
            #print(get1+"'"+str(start.value)+"'"+get2+"'"+str(end.value)+"'"+get3)
            MyURL= get1+"'"+str(self.start.value)+"'"+get2+"'"+str(self.end.value)+"'"+get3    
            return MyURL
        


    def my_QueryList(self,QueryIndex):
            #print(MyURL[1:-1])
            resp=requests.get(QueryIndex[1:-1])
            json = resp.json()
            #print(len(json['objectIds']))
            
            QueryIds=[]
            def chunks(lst, n):
                """Yield successive n-sized chunks from lst."""
                for i in range(0, len(lst), n):
                    QueryIds.append(lst[i:i + n])
            chunks(json['objectIds'],1000)
            global df
            df=pd.DataFrame([])
            max_count = len(QueryIds)
            
            f = IntProgress(min=0, max=max_count,description='APILoading:',bar_style='info') # instantiate the bar
             # display the bar
            display(f)
            for i in range(len(QueryIds)):         
                get1='"https://egis.baltimorecity.gov/egis/rest/services/GeoSpatialized_Tables/Part1_Crime/FeatureServer/0/query?where=RowID >= '
                get2= ' AND RowID <= '
                get3= '&outFields=*&returnIdsOnly=false&outSR=4326&f=json"'
                J=requests.get((get1+str(min(QueryIds[i]))+get2+str(max(QueryIds[i]))+get3)[1:-1]).json()             
                df=df.append(pd.json_normalize(J['features']),ignore_index=True)
                f.value = i+1
                if i+1 == max_count:
                    f.description = 'Complete'
                    f.style={'bar_color': 'green'}            
            #return display(f)
            colnames=list(df.columns.str.split('.').str[1])
            df.columns = colnames
            df['CrimeDate']=pd.to_datetime(df['CrimeDateTime'], unit='ms')#.dt.strftime('%Y-%m-%d')
            df.drop(['CrimeDateTime'],axis=1,inplace=True)
            df['Time'] = df.CrimeDate.dt.hour
            df['CrimeDate']=df['CrimeDate'].dt.strftime('%Y-%m-%d')
            
            #return df
            
        
    def create_expanded_button(self,description, button_style):
        return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
setup=APISetup()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
#def on_setup(btn):
#    setup.my_df(setup.get_URL())
#setup.on_click(on_setup)


def on_setup(btn):
    #outLoad.clear_output()
    with outLoad:
        setup.my_QueryList(setup.QueryIndex())
    outLoad.clear_output()
setup.on_click(on_setup)

def on_click(btn):

    with out:
        plt.rcParams["figure.figsize"] = ('16','2.9')
        df['CrimeDate']=pd.to_datetime(df['CrimeDate'])
        CrimeCount=df.groupby(['CrimeDate']).count()['CrimeCode']
        CrimeCount=CrimeCount.to_frame()
        Crim_Week = CrimeCount['CrimeCode'].resample('W').mean()
        fig, ax = plt.subplots()
        ax.plot(CrimeCount['CrimeCode'],
        marker='.', linestyle='-', linewidth=0.5, label='Daily')
        ax.plot(Crim_Week.to_frame()['CrimeCode'],
        marker='o', markersize=4, linestyle='-', label='Weekly Mean Resample')
        #ax.plot(LAodGrouped_7d.loc[start:end, 'Count'],
        #marker='.', linestyle='-', label='7-d Rolling Mean')
        ax.set_ylabel('Count')
        ax.set_title(str(setup.start.value)+' through '+str(setup.end.value)+', Reported Crime Baltimore City')
        ax.legend();
        fig.canvas.layout.width = '100%'
        fig.canvas.layout.height = '100%'
        
        fig.figsize = '100%'
        fig.tight_layout()

    out2.clear_output()
    with out2:

        if Choose.value == 'Point Map':

                mapdf=df#(df[(df['CrimeDate'] >= str(start.value)) & (df['CrimeDate'] <=str(end.value))])
                mapdf['Coordinates'] = list(zip(mapdf['Latitude'], mapdf['Longitude']))
                mapdf['CrimeDate']=mapdf['CrimeDate'].astype(str)
                %run PointConfig.py
                map_1.config = PointConfig
                map_1.add_data(data=mapdf, name='data_2')
                
                grid_options = {
                    'enableSorting': True,
                    'enableFilter': True,
                    'enableColResize': True,
                    'enableRangeSelection': True,
                }
                g=ipyaggrid.Grid(grid_data=mapdf,
                         grid_options=grid_options,
                         quick_filter=True,
                         show_toggle_edit=True,
                         export_mode="buttons",
                         export_csv=True,
                         export_excel=True,
                         theme='ag-theme-balham',
                         show_toggle_delete=True,
                         columns_fit='size_to_fit',
                         index=False,
                         keep_multiindex=False)
                display(g)                
                XX=map_1.config
                map_1.save_to_html(file_name='PopoutMap.html', data={"data_2": mapdf}, config=XX)
        elif Choose.value == 'Density Map':
                h3_level = slider.value

                def lat_lng_to_h3(row):
                    return h3.geo_to_h3(
                      row.Coordinates.y, row.Coordinates.x, h3_level)
                geo_Crime=df
                geo_Crime['Coordinates'] = list(zip(geo_Crime['Longitude'], geo_Crime['Latitude']))
                geo_Crime['Coordinates'] = geo_Crime['Coordinates'].apply(Point)
                geo_Crime = gpd.GeoDataFrame(geo_Crime, geometry='Coordinates')
                geo_Crime['h3'] = geo_Crime.apply(lat_lng_to_h3, axis=1)
                counts = geo_Crime.groupby(['h3']).h3.agg('count').to_frame('count').reset_index()
                counts=counts[counts.h3 != '0']
                def add_geometry(row):
                    points = h3.h3_to_geo_boundary(row['h3'], True)
                    return Polygon(points)

                %run HexMapConfig.py
                map_1.config = hexconfig
                map_1.add_data(data=counts, name='data_2')
                tabdf=df.filter(['h3','CrimeDate','Location','Neighborhood', 'Description','Post','Inside_Outside','Weapon'])
                grid_options = {
                    'enableSorting': True,
                    'enableFilter': True,
                    'enableColResize': True,
                    'enableRangeSelection': True,
                }
                g=ipyaggrid.Grid(grid_data=tabdf,
                         grid_options=grid_options,
                         quick_filter=True,
                         show_toggle_edit=True,
                         export_mode="buttons",
                         export_csv=True,
                         export_excel=True,
                         theme='ag-theme-balham',
                         show_toggle_delete=True,
                         columns_fit='size_to_fit',
                         #columns_fit='auto',
                         index=False,
                         keep_multiindex=False)
                display(g)
                XX=map_1.config
                map_1.save_to_html(file_name='PopoutMap.html', data={"data_2": counts}, config=XX)
        elif Choose.value == "Association Analysis":
                df.to_pickle("./df.pkl")
                %run Assoc.py
                %run ArcMapConfig.py
                #pause=3
                #df2 = pd.read_pickle("./Connections.pkl")
                df2 = pd.read_csv('connectionstest.csv')
                map_1.config = config
                map_1.add_data(data=df2, name='data_2')
                grid_options = {
                    'enableSorting': True,
                    'enableFilter': True,
                    'enableColResize': True,
                    'enableRangeSelection': True,
                }
                g=ipyaggrid.Grid(grid_data=df2.filter(['antecedentsEach','consequentsEach','support','confidence','lift','Start','Finish']),
                         grid_options=grid_options,
                         quick_filter=True,
                         show_toggle_edit=True,
                         export_mode="buttons",
                         export_csv=True,
                         export_excel=True,
                         theme='ag-theme-balham',
                         show_toggle_delete=True,
                         columns_fit='size_to_fit',
                         index=False,
                         keep_multiindex=False)
                display(g)
                XX=map_1.config
                map_1.save_to_html(file_name='PopoutMap.html', data={"data_2": df2}, config=XX)                
            
    out3.clear_output()    
    with out3:
        if Choose.value == 'Point Map':
            print(color.BOLD +"Top 3 Reported Crime Types"+color.END)
            print(df['Description'].value_counts().sort_values(ascending=False).nlargest(3).to_string(index=True, header=False))
            print('')
            print(color.BOLD +"Top 3 Crime Hours"+color.END)
            print(df.Time.value_counts().sort_values(ascending=False).nlargest(3).to_string(index=True, header=False))
            print('')
            print(color.BOLD +"Top 3 Days of the Week"+color.END)
            print(pd.DatetimeIndex(df['CrimeDate']).day_name().value_counts().sort_values(ascending=False).nlargest(3).to_string(index=True, header=False))
                
        elif Choose.value == 'Association Analysis':
            print(color.BOLD +"Top 10 Geographic Associations"+color.END)
            display(df2[0:10].filter(['antecedentsEach','consequentsEach','confidence']))
            
        elif Choose.value == 'Density Map':
            print(color.BOLD +"Top 3 Neighborhoods"+color.END)
            print(df['Neighborhood'].value_counts().sort_values(ascending=False).nlargest(3).to_string(index=True, header=False))
            print('')
            print(color.BOLD +"Top 3 Districts"+color.END)
            print(df['District'].value_counts().sort_values(ascending=False).nlargest(3).to_string(index=True, header=False))
            print('')
            print(color.BOLD +"Top 3 Location Types"+color.END)
            print(df['Premise'].value_counts().sort_values(ascending=False).nlargest(3).to_string(index=True, header=False))


outLoad = Output(layout = {
            'width': '100%',
            
            
        })
%matplotlib widget
out = Output(layout = {
            'width': '100%',
            'height': '98%',
            'border': '1px solid black'
        })
out2=Output()
out3=Output()
def dropdown_eventhandler(change):
    while Choose.value == 'Density Map' and left_button.children[-1] != slider:
        left_button.children=tuple(list(left_button.children) +[DenseLvl]+[slider])
    if Choose.value != 'Density Map' and left_button.children[-1] == slider:
        left_button.children=tuple(list(left_button.children)[:-2])

def dropdown_eventhandler2(change):
    while Choose.value == 'Association Analysis' and left_button.children[-1] != button4:
        left_button.children=tuple(list(left_button.children) +[button4])
    if Choose.value != 'Association Analysis' and left_button.children[-1] == button4:
        left_button.children=tuple(list(left_button.children)[:-1])
    elif Choose.value != 'Association Analysis' and left_button.children[-3] == button4:
        left_button.children=tuple(list(left_button.children)[0:10]+[DenseLvl]+[slider])

def on_click2(btn):
    webbrowser.open('PopoutMap.html')
def on_click4(btn):
    working_tab.children = list(working_tab.children) + [T]
    working_tab.set_title(len(working_tab.children)-1, ['Assoc Description'])
    
button = Button(description='Run', button_style='info', layout=Layout(height='auto', width='auto'))
button.on_click(on_click)
button2 = Button(description='Expand Map', button_style='info', layout=Layout(height='auto', width='auto'))
button2.on_click(on_click2)

button4 = Button(description='What is Association Analysis?', button_style='primary', layout=Layout(height='auto', width='auto'))
button4.on_click(on_click4)

Choose = Dropdown(options=[('Point Map'),('Density Map'),('Association Analysis'),],
                        button_style='info', layout=Layout(height='auto', width='auto'))

Choose.observe(dropdown_eventhandler, names='value')    
Choose.observe(dropdown_eventhandler2, names='value')    

slider = IntSlider(description='',min=3,max=10,readout=True, layout=Layout(height='auto', width='auto'))
DenseLvl = Label(value='Choose Density Level')

children=[out, out2]
titles=['Plot', 'Data']
working_tab = Tab(layout=Layout(height='405px', width='98%'))
working_tab.children = children
for i in range(len(children)):
    working_tab.set_title(i, titles[i])
    
f = open("AssocExplain.txt", "r")

T=Textarea(
    value=f.read(),
    placeholder='Type something',
    description='',
    disabled=False,
    layout=Layout(height='405px', width='100%'))
    
center_button = VBox([map_1],border='1px solid black')
left_button = VBox([Label(value="Pick a Start Date"),setup.start,Label(value="Pick a End Date"),setup.end,setup.button,button,button2,Label(value='Choose Analysis Type'),Choose,outLoad])
right_button = VBox([out3])
footer_button = VBox([working_tab], layout=Layout(height='420px', width='100%'))
AppLayout(header=setup.header_button,
          left_sidebar=left_button,
          center=center_button,
          right_sidebar=right_button,
          footer=footer_button,
          height="1100px", width="100%",grid_gap="1px")